# Runtime experiments and comparisions

This notebook explores the runtimes of our implementation with various graphs and also compares it with the implementation of the Louvain algorithm (with Modularity) in NetworkX.

Running this experiment will likely take a while, so be warned!

In [1]:
import itertools
import random
import timeit

import networkx as nx

import datasets.cora as cora, datasets.jazz as jazz, datasets.relativity as grcq

from community_detection.leiden import leiden
from community_detection.louvain import louvain
from community_detection.quality_functions import CPM, Modularity

random.seed(42)

We begin by loading our test datasets / graphs:
* The [Karate Club](https://www.journals.uchicago.edu/doi/abs/10.1086/jar.33.4.3629752?journalCode=jar) graph (34 nodes, 78 edges)
* The [Jazz Musicians](https://www.worldscientific.com/doi/abs/10.1142/S0219525903001067) graph (198 nodes, 2742 edges)
* The [Cora](https://www.openicpsr.org/openicpsr/project/100859/version/V1/view) citation graph (2708 nodes, 5278 edges)
* The [Arxiv GR-QC](http://snap.stanford.edu/data/ca-GrQc.html) citation graph (5242 nodes, 14496 edges)

In [2]:
graphs = [ nx.karate_club_graph(), jazz.get_graph(), cora.get_graph(), grcq.get_graph() ]
graph_names = [ 'Karate Club', 'Jazz Musicians', 'Cora Citations', 'Arxiv GR-QC' ]

We're also preparing the algorithms we'll be using:

* Our implementations of the Louvain and Leiden algorithms, with both Modularity and CPM as quality functions.
* The (highly optimized) [implementation](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.louvain.louvain_communities.html) of the Louvain algorithm (with Modularity) in the NetworkX library.

In [3]:
𝓗 = Modularity(1.0)

fn_louvain_mod = lambda G: louvain(G, 𝓗)
fn_leiden_mod  = lambda G: leiden(G, 𝓗)

𝓗 = CPM(0.95)
fn_louvain_cpm = lambda G: louvain(G, 𝓗)
fn_leiden_cpm  = lambda G: leiden(G, 𝓗)

fn_louvain_nwx = lambda G: nx.community.louvain_communities(G, seed=42)

algorithms = [ fn_louvain_mod, fn_leiden_mod, fn_louvain_cpm, fn_leiden_cpm, fn_louvain_nwx ]
algo_names = [ 'Louvain (Mod)', 'Leiden (Mod)', 'Louvain (CPM)', 'Leiden (CPM)', 'NetworkX' ]

In [4]:
%%time

times = {}

for ((graph, g_name), (algo, a_name)) in itertools.product(zip(graphs, graph_names), zip(algorithms, algo_names)):
    print(f"Running {a_name} on {g_name} … ", end='')
    # First, build a callable, that will repeatedly be run to measure the average execution time:
    test_callable = lambda: algo(graph)
    # Run it 10 times and take the minimum measured time (as per https://docs.python.org/3/library/timeit.html#timeit.Timer.repeat)
    time = min(timeit.repeat(stmt=test_callable, repeat=10, number=1))
    times[(algo,graph)] = time
    print(f"execution time: ~ {time:.6f} seconds.")

Running Louvain (Mod) on Karate Club … execution time: ~ 0.005921 seconds.
Running Leiden (Mod) on Karate Club … execution time: ~ 0.009455 seconds.
Running Louvain (CPM) on Karate Club … execution time: ~ 0.005198 seconds.
Running Leiden (CPM) on Karate Club … execution time: ~ 0.011457 seconds.
Running NetworkX on Karate Club … execution time: ~ 0.001248 seconds.
Running Louvain (Mod) on Jazz Musicians … execution time: ~ 0.145399 seconds.
Running Leiden (Mod) on Jazz Musicians … execution time: ~ 0.315850 seconds.
Running Louvain (CPM) on Jazz Musicians … execution time: ~ 0.172764 seconds.
Running Leiden (CPM) on Jazz Musicians … execution time: ~ 0.321889 seconds.
Running NetworkX on Jazz Musicians … execution time: ~ 0.021594 seconds.
Running Louvain (Mod) on Cora Citations … execution time: ~ 8.822755 seconds.
Running Leiden (Mod) on Cora Citations … execution time: ~ 38.115755 seconds.
Running Louvain (CPM) on Cora Citations … execution time: ~ 9.277118 seconds.
Running Leiden 

Let's print this in a nice table:

In [5]:
print(f"{'Graph'    : >14} | " + (" | ".join(f"{name: ^14}" for name in graph_names)))
print(f"{'Size n/m' : >14} | " + (" | ".join(f"{str(G.order()) + ' / ' + str(G.size()): ^14}" for G in graphs)))

for algo, algo_name in zip(algorithms, algo_names):
    print(f"{algo_name: >14}", end='')
    for G in graphs:
        time = times[(algo, graph)]
        print(f" | {time: >12.6f} s", end='')
    print()

         Graph |  Karate Club   | Jazz Musicians | Cora Citations |  Arxiv GR-QC  
      Size n/m |    34 / 78     |   198 / 2742   |  2708 / 5278   |  5242 / 14496 
 Louvain (Mod) |    27.447633 s |    27.447633 s |    27.447633 s |    27.447633 s
  Leiden (Mod) |   135.966512 s |   135.966512 s |   135.966512 s |   135.966512 s
 Louvain (CPM) |    28.742038 s |    28.742038 s |    28.742038 s |    28.742038 s
  Leiden (CPM) |   116.355158 s |   116.355158 s |   116.355158 s |   116.355158 s
      NetworkX |     0.331735 s |     0.331735 s |     0.331735 s |     0.331735 s


These tests were run on AMD Ryzen 7 PRO 5850U with 8 cores / 16 threads, running at 3.15 GHz on a system with 32 GB of RAM.